# Incremental Ernollment Trial with voxceleb1
---------

trial 길이가 충분히 긴 set을 만들고자 한다.

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pickle
import pandas as pd
from utils import key2df

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from batch_sv_system_utils import get_embeds, cosine_sim, compute_plda_score
from utils import key2df, df2dict
def get_id2idx(keys):
    key_df = key2df(keys)
    id2idx, idx2id = df2dict(key_df) 
    return id2idx

## Dataframes & embeddings

In [3]:
# voxc1_df['label'] = voxc1_df.groupby('spk').ngroup().values
# voxc1_df = voxc1_df.sort_values('id').reset_index(drop=True)
# voxc1_df = voxc1_df.fillna('test')
# voxc1_meta = pd.read_pickle("/dataset/SV_sets/voxceleb1/dataframes/voxc1_meta.pkl")
# spk2gender = voxc1_meta.Gender.to_dict()
# voxc1_df['gender'] = voxc1_df.spk.apply(lambda x: spk2gender[x])
# voxc1_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1.csv", index=False)

# voxc1_dev_df = voxc1_df[voxc1_df.spk.isin(dev_spks)]
# voxc1_eval_df = voxc1_df[voxc1_df.spk.isin(eval_spks)]
# voxc1_dev_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_dev.csv", index=False)
# voxc1_eval_df.to_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1_eval.csv", index=False)

In [4]:
voxc1_df = pd.read_csv("/dataset/SV_sets/voxceleb1/dataframes/voxc1.csv")
spk_uttr_stat = voxc1_df.spk.value_counts()
voxc1_meta = pd.read_pickle("/dataset/SV_sets/voxceleb1/dataframes/voxc1_meta.pkl")
spk2gender = voxc1_meta.Gender.to_dict()

In [124]:
embed_dir = "embeddings/voxc2_fbank64_voxc2untied_embeds/"
sv_embeds = np.load(embed_dir + "/sv_embeds.npy")
keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
id2idx = get_id2idx(keys)

plda_embed_dir = "embeddings/voxc2_fbank64_voxc2untied_xvector/"
plda_sv_embeds = np.load(plda_embed_dir + "/sv_embeds.npy")
plda_model_dir = plda_embed_dir + "plda_train/"
plda_keys = pickle.load(open(plda_embed_dir + "/sv_keys.pkl", "rb"))
plda_id2idx = get_id2idx(plda_keys)

## Hold-out Developement set

utterance가 150개 넘는 것들을 evaluation spks로 빼고 나머지는 developement spk로 빼었다.  
그리고 developement spk 음성을 이용해서 threshold를 정하기 위한 trial을 만든다.

In [76]:
dev_uttrs.head()

,file,frames,id,label,session,set,spk,gender
487,id10005/1geDB-I2TjE-00001,1326,id10005-1geDB-I2TjE-00001,4,1geDB-I2TjE,train,id10005,m
488,id10005/38_N-A-QOUo-00001,512,id10005-38_N-A-QOUo-00001,4,38_N-A-QOUo,train,id10005,m
489,id10005/38_N-A-QOUo-00002,446,id10005-38_N-A-QOUo-00002,4,38_N-A-QOUo,train,id10005,m
490,id10005/38_N-A-QOUo-00003,648,id10005-38_N-A-QOUo-00003,4,38_N-A-QOUo,train,id10005,m
491,id10005/38_N-A-QOUo-00004,613,id10005-38_N-A-QOUo-00004,4,38_N-A-QOUo,train,id10005,m


In [197]:
dev_spks = spk_uttr_stat[(spk_uttr_stat < 150)].index.tolist()
dev_uttrs = voxc1_df[voxc1_df.spk.isin(dev_spks)][['id', 'spk', 'gender']]
eval_spks = spk_uttr_stat[spk_uttr_stat >= 150].index.tolist()
eval_uttrs = voxc1_df[voxc1_df.spk.isin(eval_spks)][['id', 'spk', 'gender']]

In [198]:
print(f"number of dev spks: {len(dev_spks)}")
print(f"number of eval spks: {len(eval_spks)}")

number of dev spks: 940
number of eval spks: 311


In [200]:
dev_uttrs.to_pickle("trials/dev940_eval311/dev_uttrs.pkl")

In [288]:
from batch_sv_system_utils import compute_plda_score, get_embeds, cosine_sim
dev_uttrs = pd.read_pickle("trials/dev940_eval311/dev_uttrs.pkl")
dev_cohort_embeds = get_embeds(dev_uttrs.sample(n=5000).id, plda_sv_embeds, plda_id2idx, norm=False)
np.save("trials/dev940_eval311/dev_cohort_embeds.npy", dev_cohort_embeds)

In [7]:
dev_trials = pd.read_pickle("trials/enr306/trial_for_dev.pkl")

## Cohort Set

In [71]:
# cohort_ids = dev_uttrs.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=20)).id
# cohort_embeds = get_embeds(cohort_ids, plda_sv_embeds, plda_id2idx, norm=True)
# np.save("trials/dev317_eval934/cohort_embeds.npy", cohort_embeds)

## Random trials

###  Dev trials

In [66]:
dev_trials = []

for dev_spk in dev_spks: # TODO: multiple enrolled speakers case
    target_gender = spk2gender[dev_spk]
    target_uttrs = dev_uttrs[dev_uttrs.spk == dev_spk]
    nonTarget_uttrs = dev_uttrs[(dev_uttrs.spk != dev_spk)]
    nonTarget_uttrs = nonTarget_uttrs[nonTarget_uttrs.gender == target_gender]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=20)
    test_target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
    test_nonTarget_uttrs = nonTarget_uttrs.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=10))
    test_trial = pd.concat([test_target_uttrs, test_nonTarget_uttrs])
    dev_trials += [[dev_spk, np.array(enr_uttrs.id), 
               (np.array(test_trial.id), np.array(test_trial.label)),
             ]]
    
save_dir = "./trials/dev317_eval934/dev_random_enr20_spk10_gender/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(dev_trials, open(save_dir + "/trials.pkl", "wb"))    

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


###  Eval trials

In [82]:
eval_target_pool = eval_uttrs.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=60))
eval_nonTarget_pool = eval_uttrs.drop(index=eval_target_pool.index)

In [73]:
len(eval_nonTarget_pool.spk.unique())

934

In [64]:
eval_trials = []

for eval_spk in eval_spks: # TODO: multiple enrolled speakers case
    target_gender = spk2gender[eval_spk]
    target_uttrs = eval_target_pool[eval_target_pool.spk == eval_spk]
    nonTarget_uttrs = eval_nonTarget_pool[(eval_nonTarget_pool.spk != eval_spk)]
    nonTarget_uttrs = nonTarget_uttrs[nonTarget_uttrs.gender == target_gender]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=10)
    test_target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
#     test_nonTarget_uttrs = nonTarget_uttrs.sample(n=4950)
    test_nonTarget_uttrs = nonTarget_uttrs.groupby("spk", group_keys=False).apply(lambda x: x.sample(n=10))
    test_trial = pd.concat([target_uttrs, test_nonTarget_uttrs])
    eval_trials += [(eval_spk, np.array(enr_uttrs.id), test_trial)]
    
save_dir = "./trials/dev317_eval934/eval_random_enr10_tar50_spk10_gender/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(eval_trials, open(save_dir + "/trials.pkl", "wb"))    

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Hard trials

### Eval trial 

In [108]:
# eval_nonTarget_pool = eval_uttrs.groupby('spk', group_keys=False).apply(lambda x: x.sample(n=50))
# eval_target_pool = eval_uttrs.drop(index=eval_nonTarget_pool.index)

In [140]:
len(test_target_uttrs)

997

In [142]:
hard_neg_trials

array([35921])

In [181]:
eval_trials = []

for eval_spk in eval_spks: # TODO: multiple enrolled speakers case
    target_gender = spk2gender[eval_spk]
    target_uttrs = eval_uttrs[eval_uttrs.spk == eval_spk]
#     nonTarget_uttrs = eval_nonTarget_pool[(eval_nonTarget_pool.spk != eval_spk)]
    nonTarget_uttrs = dev_uttrs
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=6)
    test_target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
    
    enr_embeds = get_embeds(enr_uttrs.id, plda_sv_embeds, plda_id2idx)
    nonTarget_embeds = get_embeds(nonTarget_uttrs.id, plda_sv_embeds, plda_id2idx)
    scores = compute_plda_score(enr_embeds, nonTarget_embeds, plda_model_dir)
    
    for idx in range(0,6,3):
        enr_uttrs_ = enr_uttrs[idx:idx+3]
        hard_neg_trials = np.argsort(scores[idx:idx+3].mean(0), axis=0)[-len(test_target_uttrs)*9:].ravel()
        test_nonTarget_uttrs = nonTarget_uttrs.iloc[hard_neg_trials]
        test_trial = pd.concat([target_uttrs, test_nonTarget_uttrs])
        eval_trials += [(eval_spk, np.array(enr_uttrs_.id), test_trial)]
    break
    
save_dir = "./trials/eval165/eval_hard_enr10_tarAll_devNonTarget/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(eval_trials, open(save_dir + "/trials.pkl", "wb"))    

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [168]:
len(eval_trials)

2

In [132]:
eval_trials[0]

('id10986', array(['id10986-lFEBeMFuL_M-00070', 'id10986-w7DmOHDIMQ8-00010',
        'id10986-6SBGrr4iFp0-00032', 'id10986-w7DmOHDIMQ8-00013',
        'id10986-Vzqmi3MAixQ-00118'], dtype=object),                                id      spk gender  label
 119073  id10986-33NaEPsiFEA-00001  id10986      m      1
 119074  id10986-33NaEPsiFEA-00002  id10986      m      1
 119075  id10986-33NaEPsiFEA-00003  id10986      m      1
 119076  id10986-33NaEPsiFEA-00004  id10986      m      1
 119077  id10986-4p9Fm05hlIM-00001  id10986      m      1
 119078  id10986-4p9Fm05hlIM-00002  id10986      m      1
 119079  id10986-4p9Fm05hlIM-00003  id10986      m      1
 119080  id10986-4p9Fm05hlIM-00004  id10986      m      1
 119081  id10986-4p9Fm05hlIM-00005  id10986      m      1
 119082  id10986-4p9Fm05hlIM-00006  id10986      m      1
 119083  id10986-4p9Fm05hlIM-00007  id10986      m      1
 119084  id10986-4p9Fm05hlIM-00008  id10986      m      1
 119085  id10986-4p9Fm05hlIM-00009  id10986      m 

In [186]:
enr_embeds = get_embeds(eval_trials[1][1], plda_sv_embeds, plda_id2idx)
test_embeds = get_embeds(eval_trials[1][2].id, plda_sv_embeds, plda_id2idx)
scores = compute_plda_score(enr_embeds, test_embeds, plda_model_dir)

In [187]:
from batch_sv_system_utils import compute_eer
compute_eer(scores.mean(0), eval_trials[0][2].label)

(0.026137818072021965,
 0.026327532351628737,
 0.02594810379241519,
 20.468533333333333)

In [130]:
np.sort(scores.mean(0))[-8100]

0.5455493

In [131]:
scores.min()

0.15912044

In [122]:
test_target_uttrs.shape

(947, 4)

In [112]:
np.unique(hard_neg_trials).shape

(224,)

In [120]:
eval_trials = []

for eval_spk in eval_spks: # TODO: multiple enrolled speakers case
    target_uttrs = eval_uttrs[eval_uttrs.spk == eval_spk]
    target_gender = spk2gender[eval_spk]
#     nonTarget_uttrs = eval_uttrs[(eval_uttrs.spk != eval_spk) & (eval_uttrs.gender == target_gender)]
    nonTarget_uttrs = eval_uttrs[(eval_uttrs.spk != eval_spk)]
    target_uttrs.loc[:, 'label'] = 1
    nonTarget_uttrs.loc[:, 'label'] = 0
    
    enr_uttrs = target_uttrs.sample(n=10)
    target_uttrs = target_uttrs.drop(index=enr_uttrs.index)
    
    enr_embeds = get_embeds(enr_uttrs.id, sv_embeds, id2idx)
    nonTarget_embeds = get_embeds(nonTarget_uttrs.id, sv_embeds, id2idx)
    scores = cosine_sim(enr_embeds, nonTarget_embeds)
    hard_neg_trials = np.argsort(scores, axis=1)[:, -100:].ravel()
    
    test_nonTarget_uttrs = nonTarget_uttrs.iloc[hard_neg_trials]
    imposters = test_nonTarget_uttrs.spk.unique().tolist()
    test_trial = pd.concat([target_uttrs, test_nonTarget_uttrs]).sample(frac=1)
    eval_trials += [[eval_spk, imposters, enr_uttrs.id.tolist(), 
               (np.array(test_trial.id),np.array(test_trial.label)),
             ]]

save_dir = "./trials/enr306/eval_hard_enr10_n1000/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
pickle.dump(eval_trials, open(save_dir + "/trials.pkl", "wb"))    

/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
